# Notebook

Analysis of velocity fields obtained from flow MRI in ten patients with TAVI implants.

## Setup and imports

Configure and import all necessary libraries for model visualization.

In [1]:
# Standard library imports
import os
from pathlib import Path

# Third-party imports
import pyvista as pv
from rich import print

# Local imports
import magflow.utils.visualization as visualization

# For notebook integration
pv.set_jupyter_backend("server")
%matplotlib inline

## Dataset Loading

Se cargan los datos en formato `vtk` de los campos de velocidades del paciente. Por defecto, se carga el instante de tiempo 85 dado que coincide con el instante de sístole y, por tanto, presenta mayor contraste.

In [2]:
# Parameters
data_dir = Path("../../data")
timestep = 85  # Default timestep (systole phase)
biomodel_path = Path("../../assets/biomodel.vtk")

# List available timesteps
available_files = [f for f in os.listdir(data_dir) if f.startswith("data.vts.")]
available_timesteps = [int(f.split(".")[-1]) for f in available_files]
available_timesteps.sort()

print(f"Available timesteps: {available_timesteps}")
print(f"Selected timestep: {timestep}")

# Construct the full filepath for the selected timestep data
filename = f"data.vts.{timestep}"
filepath = data_dir / filename

# Validate that the requested timestep file exists
if not filepath.exists():
    print(f"[bold]Error:[/bold] File {filepath} not found.")
    print(f"Please select from available timesteps: {available_timesteps}")
else:
    # Load the VTS dataset for the selected timestep
    dataset = pv.read(str(filepath), force_ext=".vts")
    print(f"Loaded velocity field dataset (timestep {timestep}) from {filepath}")

    # Load the biomodel data
    biomodel_data = visualization.load_biomodel(biomodel_path)
    print(f"Loaded anatomical biomodel from {biomodel_path}")
    print(
        f"Biomodel contains {biomodel_data.n_points} points and {biomodel_data.n_cells} cells"
    )
    print(f"Velocity field dimensions: {dataset.dimensions}")

Available timesteps: [0, 28, 57, 85, 113, 141, 170, 198, 226, 254, 283, 311, 339, 368, 396, 424, 452, 481, 509, 
537, 566, 594, 622, 650]

Selected timestep: 85

Loaded velocity field dataset (timestep 85) from ..\..\data\data.vts.85

Applied rotations: Y-axis: -90°, Z-axis: -90°

Translated model: Y-axis: +300 units

Mirrored biomodel in XY plane

Loaded anatomical biomodel from ..\..\assets\biomodel.vtk

Biomodel contains 5174 points and 10344 cells

Velocity field dimensions: (128, 128, 40)

## Masking

Intersección entre el campo de velocidades y la geometría aórtica. De esta forma se eliminan todos los valores de velocidad fuera de la región aórtica. También se calculan estadísticas generales:

- Velocidad media [cm/s]
- Velocidad máxima [cm/s]

In [3]:
# Load data files
try:
    # Extract aorta region from dataset
    aorta = visualization.extract_aorta(dataset, biomodel_data)

    # Calculate and display velocity statistics for the current timestep
    velocity_stats = visualization.calculate_velocity_statistics(aorta)

    # Display velocity statistics using standard prints instead of rich table
    print("Velocity Statistics:")
    print(f"Mean Velocity: {velocity_stats['mean']:.2f} cm/s")
    print(f"Peak Velocity: {velocity_stats['peak']:.2f} cm/s")

except Exception as e:
    print(f"Error loading data: {e}")

Velocity Statistics:

Mean Velocity: 29.15 cm/s

Peak Velocity: 195.32 cm/s

## Velocity Analysis

Visualización del campo de velocidades del paciente e instante de tiempo seleccionado.

Los valores representan la magnitud del vector velocidad.

$$
V = \sqrt{\vec{u}^2 + \vec{v}^2 + \vec{w}^2}
$$

También se ha limitado el rango de valores [0,150] cm/s.

Queda pendiente modificar el sistema de coordenadas para visualizar la aorta en posición vertical

In [ ]:
# Initialize the PyVista plotter for 3D visualization
velocity_plotter = pv.Plotter(notebook=True)

volume = visualization.render_volume(aorta)  # Create volume representation of the aorta

# Add biomodel as a wireframe to provide anatomical context
velocity_plotter.add_mesh(
    biomodel_data,
    color="white",
    opacity=0.8,
    label="Biomodel",
    show_edges=True,
    edge_color="black",
    style="wireframe",
)

# Add volume rendering of velocity magnitude
volume_plot = velocity_plotter.add_volume(
    volume,
    scalars="VelocityMagnitude",
    clim=[0, 150],
    opacity="linear",
    scalar_bar_args=dict(
        title="Velocity [cm/s]",
        n_labels=6,
        vertical=True,
    ),
    mapper="smart",  # Smart volume mapper for better rendering
    cmap="rainbow",  # Color map for velocity visualization
    blending="composite",  # Composite blending mode for volume rendering
)

# Configure camera and grid for optimal viewing
velocity_plotter.view_isometric()  # Set isometric view
velocity_plotter.show_grid(font_size=12)
velocity_plotter.add_axes()

# Render the scene
velocity_plotter.show()

### Líneas de flujo

In [5]:
# Initialize the PyVista plotter for 3D visualization
streamlines_plotter = pv.Plotter(notebook=True)

# Define center point for streamline generation (coordinates in the dataset space)
center = [130, 260, 40]
streamlines, source = visualization.generate_streamlines(aorta, center)

# Add biomodel as a wireframe to provide anatomical context
streamlines_plotter.add_mesh(
    biomodel_data,
    color="white",
    opacity=0.8,
    label="Biomodel",
    show_edges=True,
    edge_color="black",
    style="wireframe",
)

# Add streamlines visualization colored by velocity magnitude
streamlines_actor = streamlines_plotter.add_mesh(
    streamlines,
    line_width=2.0,
    label="Streamlines",
    scalars="VelocityMagnitude",  # Use velocity magnitude for coloring
    cmap="rainbow",  # Use rainbow colormap
    clim=[0, 150],  # Consistent with volume visualization
    scalar_bar_args=dict(
        title="Velocity [cm/s]",
        n_labels=6,
        vertical=True,
    ),
)

# Add visualization of streamline seed points
source_actor = streamlines_plotter.add_mesh(
    source,
    color="yellow",
    point_size=8,
    render_points_as_spheres=True,
    label="Seed Points",
)

# Configure camera and grid for optimal viewing
streamlines_plotter.view_isometric()  # Set isometric view
streamlines_plotter.show_grid(font_size=12)
streamlines_plotter.add_axes()

streamlines_plotter.show()

Generating streamlines from velocity field

Widget(value='<iframe src="http://localhost:62563/index.html?ui=P_0x1da7831b980_1&reconnect=auto" class="pyvis…